# Hidden Markov Models


이 노트북은 순차 데이터에 대한 간단한 모델인 HMM(Hidden Markov Model)을 소개합니다.

다음 내용을 다룹니다.:
- HMM이란 무엇이며 언제 사용하고 싶은지
- HMM의 "3가지 문제"
- PyTorch에서 HMM을 구현하는 방법.

(이 노트북의 코드는 https://github.com/lorenlugosch/pytorch_HMM에서도 찾을 수 있습니다.)

가상 시나리오
------

HMM 사용에 동기를 부여하려면 여행을 많이 하는 친구가 있다고 상상해 보십시오. 제트기 친구는 매일 자신이 있는 도시에서 셀카를 보내 당신을 부러워하게 만듭니다.

<center>

![Diagram of a traveling friend sending selfies](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/selfies.png?raw=true)
</center>





셀카만 보고 매일 그 친구가 어느 도시에 있는지 추측하는 방법은 무엇입니까?

셀카에 에펠탑과 같이 정말 눈에 띄는 랜드마크가 포함되어 있으면 사진을 찍은 위치를 쉽게 파악할 수 있습니다. 그렇지 않으면 도시를 유추하는 것이 훨씬 더 어려울 것입니다.

그러나 우리에게는 도움이 될 단서가 있습니다. 친구가 매일 있는 도시는 완전히 무작위가 아닙니다. 예를 들어 친구는 새로운 도시로 날아가기 전에 관광을 위해 며칠 동안 같은 도시에 머물 것입니다.

## The HMM setup

친구가 도시를 오가며 셀카를 보내는 가상 시나리오는 HMM을 사용하여 모델링할 수 있습니다.


HMM은 주어진 시간에 특정 상태에 있는 시스템을 모델링하고 해당 상태에 따라 출력을 생성합니다.

각 타임스텝 또는 클록 틱에서 시스템은 무작위로 새 상태를 결정하고 해당 상태로 이동합니다. 그런 다음 시스템은 무작위로 관찰을 생성합니다. 상태는 "숨겨져" 있으므로 관찰할 수 없습니다. (도시/셀카 비유에서 미지의 도시는 숨겨진 상태가 되고 셀카는 관찰 대상이 됩니다.)

상태의 시퀀스를 $\mathbf{z} = \{z_1, z_2, \dots, z_T \}$로 나타내자. 여기서 각 상태는 유한한 $N$ 상태 집합 중 하나이고 관찰 시퀀스는 $로 나타냅니다. \mathbf{x} = \{x_1, x_2, \dots, x_T\}$. 관측값은 문자처럼 불연속적이거나 오디오 프레임처럼 실제 값일 수 있습니다.

<center>

![Diagram of an HMM for three timesteps](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/hmm.png?raw=true)
</center>

HMM은 두 가지 주요 가정을 합니다.
- **가정 1:** 시간 $t$의 상태는 이전 시간 $t-1$의 상태에만 의존합니다.
- **가정 2:** $t$ 시간의 출력은 $t$ 시간의 상태에만 의존합니다.

이 두 가지 가정을 통해 우리가 관심을 가질 수 있는 특정 수량을 효율적으로 계산할 수 있습니다.

## HMM의 구성 요소
HMM에는 세 가지 학습 가능한 매개변수 세트가 있습니다.
  


- **전환 모델**은 정사각형 행렬 $A$이며 $A_{s, s'}$는 $p(z_t = s|z_{t-1} = s')$, 점프 확률을 나타냅니다. 상태 $s'$에서 상태 $s$로.

- **방출 모델** $b_s(x_t)$는 시스템이 $s$ 상태에 있을 때 $x_t$를 생성할 확률인 $p(x_t|z_t = s)$를 알려줍니다. 이 노트북에서 사용할 불연속 관측의 경우 방출 모델은 각 상태에 대해 하나의 행과 각 관측에 대해 하나의 열이 있는 조회 테이블일 뿐입니다. 실제 값 관측의 경우 방출 모델을 구현하기 위해 가우시안 혼합 모델 또는 신경망을 사용하는 것이 일반적입니다.

- **상태 사전**은 $s$ 상태에서 시작할 확률인 $p(z_1 = s)$를 알려줍니다. 우리는 $\pi$를 사용하여 상태 사전 벡터를 나타내므로 $\pi_s$는 상태 $s$에 대한 사전 상태입니다.

PyTorch에서 HMM 클래스를 프로그래밍해 봅시다.

In [3]:
import torch
import numpy as np

a = np.array([[1.,1.],
              [2.,2.]])
b = torch.tensor(a)
print(b)
c = torch.nn.Parameter(b)
print(c)

tensor([[1., 1.],
        [2., 2.]], dtype=torch.float64)
Parameter containing:
tensor([[1., 1.],
        [2., 2.]], dtype=torch.float64, requires_grad=True)


In [4]:
import torch
import numpy as np

a = torch.randn(2,2)
b = torch.nn.Parameter(a)
print(b)

Parameter containing:
tensor([[-0.2913,  0.4814],
        [ 0.2206, -0.6649]], requires_grad=True)


In [5]:
import torch
import numpy as np

class HMM(torch.nn.Module):
  """
  Hidden Markov Model with discrete observations.
  """
  def __init__(self, M, N):
    super(HMM, self).__init__()
    self.M = M # number of possible observations
    self.N = N # number of states

    # A
    self.transition_model = TransitionModel(self.N)

    # b(x_t)
    self.emission_model = EmissionModel(self.N,self.M)

    # pi
    self.unnormalized_state_priors = torch.nn.Parameter(torch.randn(self.N))

    # use the GPU
    self.is_cuda = torch.cuda.is_available()
    if self.is_cuda: self.cuda()

class TransitionModel(torch.nn.Module):
  def __init__(self, N):
    super(TransitionModel, self).__init__()
    self.N = N
    self.unnormalized_transition_matrix = torch.nn.Parameter(torch.randn(N,N))

class EmissionModel(torch.nn.Module):
  def __init__(self, N, M):
    super(EmissionModel, self).__init__()
    self.N = N
    self.M = M
    self.unnormalized_emission_matrix = torch.nn.Parameter(torch.randn(N,M))

HMM에서 샘플링하기 위해 상태 이전 분포에서 임의의 초기 상태를 선택하여 시작합니다.

그런 다음 방출 분포에서 출력을 샘플링하고 전이 분포에서 전이를 샘플링하고 반복합니다.

(정규화되지 않은 모델 파라미터를 softmax 함수를 통해 전달하여 확률로 만듭니다.)


In [6]:
a = torch.tensor([1.,2.,3.,4.,5.])
b = torch.nn.functional.softmax(a, dim=0)
print(b, b.sum())

tensor([0.0117, 0.0317, 0.0861, 0.2341, 0.6364]) tensor(1.)


In [7]:
a = torch.tensor([[1.,2.,3.,4.],
                  [1.,2.,3.,4.],
                  [1.,2.,3.,-np.inf]])
b = torch.nn.functional.softmax(a, dim=1)
print(b)

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439],
        [0.0900, 0.2447, 0.6652, 0.0000]])


In [8]:
a = torch.tensor([[1.,2.,3.,4.],
                  [1.,2.,3.,4.],
                  [1.,2.,3.,4.]])
b = torch.nn.functional.softmax(a, dim=0)
print(b)

tensor([[0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333, 0.3333]])


In [9]:
m = torch.distributions.categorical.Categorical(torch.tensor([ 0.9, 0.1, 0.0, 0.0 ]))
m.sample().item()

1

In [10]:
a = torch.tensor([ [0.8, 0.3], 
                   [0.2, 0.7] ])
print(a[:,1])
m = torch.distributions.categorical.Categorical(a[:,1])
m.sample().item()

tensor([0.3000, 0.7000])


1

In [11]:
def sample(self, T=10):
  state_priors = torch.nn.functional.softmax(self.unnormalized_state_priors, dim=0)
  transition_matrix = torch.nn.functional.softmax(self.transition_model.unnormalized_transition_matrix, dim=0)
  emission_matrix = torch.nn.functional.softmax(self.emission_model.unnormalized_emission_matrix, dim=1)

  # sample initial state
  z_t = torch.distributions.categorical.Categorical(state_priors).sample().item()
  z = []; x = []
  z.append(z_t)
  for t in range(0,T):
    # sample emission
    x_t = torch.distributions.categorical.Categorical(emission_matrix[z_t]).sample().item()
    x.append(x_t)

    # sample transition
    z_t = torch.distributions.categorical.Categorical(transition_matrix[:,z_t]).sample().item()
    if t < T-1: z.append(z_t)

  return x, z

# Add the sampling method to our HMM class
HMM.sample = sample

가짜 단어를 생성하기 위해 HMM을 하드 코딩해 봅시다. (문자열 인코딩 및 디코딩을 위한 몇 가지 도우미 함수도 추가합니다.)

우리는 시스템이 모음을 생성하기 위한 하나의 상태와 자음을 생성하기 위한 하나의 상태를 가지고 있고, 전환 행렬이 대각선에 0을 가지고 있다고 가정할 것입니다. 한 단계; 전환해야합니다.

softmax를 통해 전이 행렬을 전달하므로 0을 얻기 위해 비정규화 매개변수 값을 $-\infty$로 설정합니다.

In [12]:
import string
alphabet = string.ascii_lowercase
print(alphabet, len(alphabet))

abcdefghijklmnopqrstuvwxyz 26


In [13]:
print(alphabet.index('a'))
print(alphabet[0])

0
a


In [14]:
a = torch.tensor([alphabet.index(letter) for letter in "aeiou"])
print(a)

tensor([ 0,  4,  8, 14, 20])


In [15]:
b = torch.tensor([alphabet.index(letter) for letter in "bcdfghjklmnpqrstvwxyz"])
print(b)

tensor([ 1,  2,  3,  5,  6,  7,  9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22,
        23, 24, 25])


In [16]:
import string
alphabet = string.ascii_lowercase

def encode(s):
  """
  Convert a string into a list of integers
  """
  x = [alphabet.index(ss) for ss in s]
  return x

def decode(x):
  """
  Convert list of ints to string
  """
  s = "".join([alphabet[xx] for xx in x])
  return s

# Initialize the model
model = HMM(M=len(alphabet), N=2) 

# for p in model.parameters():
#     print(p)
# Hard-wiring the parameters!
# Let state 0 = consonant, state 1 = vowel
for p in model.parameters():
    p.requires_grad = False # needed to do lines below
model.unnormalized_state_priors[0] = 0.    # Let's start with a consonant more frequently
model.unnormalized_state_priors[1] = -0.5
print("State priors:", torch.nn.functional.softmax(model.unnormalized_state_priors, dim=0))

# In state 0, only allow consonants; in state 1, only allow vowels
vowel_indices = torch.tensor([alphabet.index(letter) for letter in "aeiou"])
consonant_indices = torch.tensor([alphabet.index(letter) for letter in "bcdfghjklmnpqrstvwxyz"])
model.emission_model.unnormalized_emission_matrix[0, vowel_indices] = -np.inf
model.emission_model.unnormalized_emission_matrix[1, consonant_indices] = -np.inf 
print("Emission matrix:", torch.nn.functional.softmax(model.emission_model.unnormalized_emission_matrix, dim=1))

# Only allow vowel -> consonant and consonant -> vowel

model.transition_model.unnormalized_transition_matrix[0,0] = -np.inf  # consonant -> consonant
model.transition_model.unnormalized_transition_matrix[0,1] = 0.       # vowel -> consonant
model.transition_model.unnormalized_transition_matrix[1,0] = 0.       # consonant -> vowel
model.transition_model.unnormalized_transition_matrix[1,1] = -np.inf  # vowel -> vowel
print("Transition matrix:", torch.nn.functional.softmax(model.transition_model.unnormalized_transition_matrix, dim=0))



State priors: tensor([0.6225, 0.3775], device='cuda:0')
Emission matrix: tensor([[0.0000, 0.0441, 0.0333, 0.0553, 0.0000, 0.0320, 0.0787, 0.0235, 0.0000,
         0.1166, 0.0154, 0.0087, 0.0333, 0.0160, 0.0000, 0.0457, 0.0311, 0.0315,
         0.0582, 0.0178, 0.0000, 0.1285, 0.0220, 0.1076, 0.0240, 0.0769],
        [0.1393, 0.0000, 0.0000, 0.0000, 0.1621, 0.0000, 0.0000, 0.0000, 0.0331,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3359, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.3296, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
Transition matrix: tensor([[0., 1.],
        [1., 0.]], device='cuda:0')


하드 코딩된 모델에서 샘플링을 시도하십시오.


In [17]:
# Sample some outputs
for _ in range(4):
  sampled_x, sampled_z = model.sample(T=5)
  print("x:", decode(sampled_x))
  print("z:", sampled_z)

x: omuxu
z: [1, 0, 1, 0, 1]
x: agavu
z: [1, 0, 1, 0, 1]
x: xugug
z: [0, 1, 0, 1, 0]
x: yegor
z: [0, 1, 0, 1, 0]


## 세 가지 문제

HMM에 대한 [클래식 자습서](https://www.cs.cmu.edu/~cga/behavior/rabiner1.pdf)에서 Lawrence Rabiner는 HMM을 효과적으로 사용하기 전에 해결해야 하는 "세 가지 문제"를 설명합니다. . 그들은:
- 문제 1: $p(\mathbf{x})$를 어떻게 효율적으로 계산합니까?
- 문제 2: 데이터를 생성했을 가능성이 가장 높은 상태 시퀀스 $\mathbf{z}$를 어떻게 찾습니까?
- 문제 3: 모델을 어떻게 교육합니까?

노트북의 나머지 부분에서는 각 문제를 해결하고 PyTorch에서 솔루션을 구현하는 방법을 살펴보겠습니다.

### 문제 1: $p(\mathbf{x})$를 어떻게 계산합니까?

#### *왜?*
$p(\mathbf{x})$ 컴퓨팅에 관심을 갖는 이유는 무엇입니까? 두 가지 이유가 있습니다.
* 두 개의 HMM $\theta_1$ 및 $\theta_2$가 주어지면 각 모델 $p_{\theta_1}(\mathbf{x})$ 및 $p_{\theta_2}(\mathbf{x})$에서 데이터의 가능성을 계산할 수 있습니다.     
둘중 데이터에 더 적합한 모델을 결정합니다.

   (예를 들어, 영어 음성에 대한 HMM과 프랑스어 음성에 대한 HMM이 주어지면 각 모델에 대한 가능성을 계산하고 그 사람이 영어를 말하는지 프랑스어를 말하는지 추론할 가능성이 더 높은 모델을 선택할 수 있습니다.)
* 나중에 살펴보겠지만 $p(\mathbf{x})$를 계산할 수 있으면 모델을 훈련할 수 있습니다.

#### *어떻게?*
우리가 $p(\mathbf{x})$를 원한다고 가정하면 어떻게 계산할까요?

우리는 $\mathbf{z}$ 상태의 일부 시퀀스를 방문하고 방출 분포 $p(x_t|z_t)$에서 각 $z_t$에 대한 출력 $x_t$를 선택하여 데이터가 생성되었다고 가정했습니다. 따라서 $\mathbf{z}$를 안다면 $\mathbf{x}$의 확률은 다음과 같이 계산할 수 있습니다.

$$p(\mathbf{x}|\mathbf{z}) = \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

그러나 우리는 $\mathbf{z}$를 모릅니다. 숨겨져 있습니다. 그러나 우리는 우리가 관찰한 것과는 별개로 주어진 $\mathbf{z}$의 확률을 알고 있습니다. 따라서 다음과 같이 $\mathbf{z}$에 대한 다양한 가능성을 합산하여 $\mathbf{x}$의 확률을 얻을 수 있습니다.

$$p(\mathbf{x}) = \sum_{\mathbf{z}} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z}) = \sum_{\mathbf{z }} \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

문제는 이 합계를 직접 계산하려면 $N^T$ 항을 계산해야 한다는 것입니다. 이것은 매우 짧은 시퀀스 외에는 불가능합니다. 예를 들어 시퀀스의 길이가 $T=100$이고 가능한 상태가 $N=2$라고 가정해 보겠습니다. 그런 다음 $N^T = 2^{100} \approx 10^{30}$ 다른 가능한 상태 시퀀스를 확인해야 합니다.

모든 $N^T$ 항을 명시적으로 계산할 필요가 없는 $p(\mathbf{x})$를 계산하는 방법이 필요합니다. 이를 위해 순방향 알고리즘을 사용합니다.

________

<u><b>The Forward Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha_{s,1} := b_s(x_1) \cdot \pi_s$ 
> 
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\alpha_{s,t} := b_s(x_t) \cdot \underset{s'}{\sum} A_{s, s'} \cdot \alpha_{s',t-1} $
> 
> $p(\mathbf{x}) := \underset{s}{\sum} \alpha_{s,T}$\
> return $p(\mathbf{x})$
________


순방향 알고리즘은 가능한 모든 $N^T$ 상태 시퀀스를 열거하는 것보다 훨씬 빠릅니다. 각 단계는 대부분 순방향 변수의 벡터에 전이 행렬을 곱하기 때문에 실행하는 데 $O(N^2T)$ 작업만 필요합니다. (그리고 전환 행렬이 희박한 경우 매우 자주 복잡성을 훨씬 더 줄일 수 있습니다.)

위에 제시된 순방향 알고리즘에는 한 가지 실질적인 문제가 있습니다. 확률은 항상 0과 1 사이이기 때문에 작은 숫자의 긴 체인을 곱하기 때문에 언더플로가 발생하기 쉽습니다. 대신 로그 도메인에서 모든 작업을 수행하겠습니다. 로그 도메인에서 곱셈은 합이 되고, 합은 [logsumexp](https://lorenlugosch.github.io/posts/2020/06/logsumexp/)가 됩니다.

________

<u><b>The Forward Algorithm (Log Domain)</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\text{log }\alpha_{s,1} := \text{log }b_s(x_1) + \text{log }\pi_s$ 
> 
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\text{log }\alpha_{s,t} := \text{log }b_s(x_t) +  \underset{s'}{\text{logsumexp}} \left( \text{log }A_{s, s'} + \text{log }\alpha_{s',t-1} \right)$
> 
> $\text{log }p(\mathbf{x}) := \underset{s}{\text{logsumexp}} \left( \text{log }\alpha_{s,T} \right)$\
> return $\text{log }p(\mathbf{x})$
________

이제 정방향 알고리즘의 수치적으로 안정적인 버전이 있으므로 PyTorch에서 구현해 보겠습니다.

In [18]:
a = torch.tensor([1,2,3])
print(a.max())
b = torch.logsumexp(a, dim=0)
print(b)

tensor(3)
tensor(3.4076)


In [56]:
a = np.array([[1,2,3],
              [4,5,6]])

# a[0].shape
# a[:,2]
print(a[:,[2]])                 # (2,1)
print(a[:,[2]].transpose(1,0))  # (1,2)   

[[3]
 [6]]
[[3 6]]


In [55]:
a = torch.tensor([[1,2,3],
                  [4,5,6]])

# a[0].shape
# a[:,2]
print(a[:,[2]])                 # (2,1)
print(a[:,[2]].transpose(1,0))  # (1,2)     

tensor([[3],
        [6]])
tensor([[3, 6]])


In [102]:
def HMM_forward(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)

  Compute log p(x) for each example in the batch.
  T = length of each example
  """
  if self.is_cuda:
  	x = x.cuda()
  	T = T.cuda()

  batch_size = x.shape[0]; T_max = x.shape[1]
  log_state_priors = torch.nn.functional.log_softmax(self.unnormalized_state_priors, dim=0)
#   print(log_state_priors)
  log_alpha = torch.zeros(batch_size, T_max, self.N)
#   print(log_alpha.shape)
  if self.is_cuda: log_alpha = log_alpha.cuda()

  log_alpha[:, 0, :] = self.emission_model(x[:,0]) + log_state_priors
#   print(log_alpha)
  for t in range(1, T_max):
    log_alpha[:, t, :] = self.emission_model(x[:,t]) + self.transition_model(log_alpha[:, t-1, :])
    # print(log_alpha)

#   print(log_alpha)
  # Select the sum for the final timestep (each x may have different length).
  log_sums = log_alpha.logsumexp(dim=2)
#   print("log_sums", log_sums)
  log_probs = torch.gather(log_sums, 1, T.view(-1,1) - 1)
  return log_probs

def emission_model_forward(self, x_t):
#   print("emission_model_forward", x_t)
  log_emission_matrix = torch.nn.functional.log_softmax(self.unnormalized_emission_matrix, dim=1)
#   print(log_emission_matrix)
  out = log_emission_matrix[:, x_t].transpose(1,0)
  return out

def transition_model_forward(self, log_alpha):
  """
  log_alpha : Tensor of shape (batch size, N)
  Multiply previous timestep's alphas by transition matrix (in log domain)
  """
  log_transition_matrix = torch.nn.functional.log_softmax(self.unnormalized_transition_matrix, dim=0)
#   print("log_transition_matrix", log_transition_matrix)
  # Matrix multiplication in the log domain
  out = log_domain_matmul(log_transition_matrix, log_alpha.transpose(0,1)).transpose(0,1)
  return out

def log_domain_matmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Normally, a matrix multiplication
	computes out_{i,j} = sum_k A_{i,k} x B_{k,j}

	A log domain matrix multiplication
	computes out_{i,j} = logsumexp_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	# log_A_expanded = torch.stack([log_A] * p, dim=2)
	# log_B_expanded = torch.stack([log_B] * m, dim=0)
    # fix for PyTorch > 1.5 by egaznep on Github:
	log_A_expanded = torch.reshape(log_A, (m,n,1))
	log_B_expanded = torch.reshape(log_B, (1,n,p))

	elementwise_sum = log_A_expanded + log_B_expanded
	out = torch.logsumexp(elementwise_sum, dim=1)

	return out

TransitionModel.forward = transition_model_forward
EmissionModel.forward = emission_model_forward
HMM.forward = HMM_forward

이전의 모음/자음 모델에서 순방향 알고리즘을 실행해 보세요.

In [67]:
encode("cat")

[2, 0, 19]

In [68]:
x = torch.stack( [torch.tensor(encode("cat"))] )
print(x)
T = torch.tensor([3])
# print(T)
print(model.forward(x, T))

tensor([[ 2,  0, 19]])
tensor([-0.4741, -0.9741], device='cuda:0')
emission_model_forward tensor([2], device='cuda:0')
tensor([[   -inf, -3.1207, -3.4031, -2.8951,    -inf, -3.4414, -2.5423, -3.7510,
            -inf, -2.1494, -4.1715, -4.7487, -3.4030, -4.1364,    -inf, -3.0860,
         -3.4701, -3.4584, -2.8435, -4.0296,    -inf, -2.0517, -3.8177, -2.2294,
         -3.7313, -2.5654],
        [-1.9711,    -inf,    -inf,    -inf, -1.8195,    -inf,    -inf,    -inf,
         -3.4084,    -inf,    -inf,    -inf,    -inf,    -inf, -1.0908,    -inf,
            -inf,    -inf,    -inf,    -inf, -1.1100,    -inf,    -inf,    -inf,
            -inf,    -inf]], device='cuda:0')
tensor([[[-3.8771,    -inf],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]], device='cuda:0')
emission_model_forward tensor([0], device='cuda:0')
tensor([[   -inf, -3.1207, -3.4031, -2.8951,    -inf, -3.4414, -2.5423, -3.7510,
            -inf, -2.1494, -4.1715, -4.7487, -3.4030, -4.1364,    -inf, -3.0860,
 

In [69]:
x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
print(x)
T = torch.tensor([3,3])
print(model.forward(x, T))

tensor([[0, 1, 0],
        [0, 1, 1]])
tensor([-0.4741, -0.9741], device='cuda:0')
emission_model_forward tensor([0, 0], device='cuda:0')
tensor([[   -inf, -3.1207, -3.4031, -2.8951,    -inf, -3.4414, -2.5423, -3.7510,
            -inf, -2.1494, -4.1715, -4.7487, -3.4030, -4.1364,    -inf, -3.0860,
         -3.4701, -3.4584, -2.8435, -4.0296,    -inf, -2.0517, -3.8177, -2.2294,
         -3.7313, -2.5654],
        [-1.9711,    -inf,    -inf,    -inf, -1.8195,    -inf,    -inf,    -inf,
         -3.4084,    -inf,    -inf,    -inf,    -inf,    -inf, -1.0908,    -inf,
            -inf,    -inf,    -inf,    -inf, -1.1100,    -inf,    -inf,    -inf,
            -inf,    -inf]], device='cuda:0')
tensor([[[   -inf, -2.9452],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[   -inf, -2.9452],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]], device='cuda:0')
emission_model_forward tensor([1, 1], device='cuda:0')
tensor([[   -inf, -3.1207, -3.4031, -2.8951,    -inf

위에서 모음 <-> 자음 HMM을 사용할 때 정방향 알고리즘은 $\mathbf{x} = \text{"abb"}$에 대해 $-\infty$를 반환합니다. 전이 행렬이 모음 -> 모음과 자음 -> 자음의 확률이 0이라고 말하므로 $\text{"abb"}$가 발생할 확률은 0이므로 로그 확률은 $-\infty$입니다.

#### *참고: 정방향 알고리즘 도출*

순방향 알고리즘이 실제로 $p(\mathbf{x})$를 계산하는 방법을 이해하는 데 관심이 있다면 이 섹션을 읽으십시오. 그렇지 않은 경우 "문제 2"의 다음 부분(가장 가능성이 높은 상태 시퀀스 찾기)으로 건너뜁니다.



forward algorithm을 도출하려면 먼저 forward variable 를 도출합니다.

$
\begin{align} 
    \alpha_{s,t} &= p(x_1, x_2, \dots, x_t, z_t=s) \\
     &= p(x_t | x_1, x_2, \dots, x_{t-1}, z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s)  \\ 
    &= p(x_t | z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s) \\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s', z_t = s) \right)\\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(z_t = s | x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \cdot p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \right)\\
    &= \underbrace{p(x_t | z_t = s)}_{\text{emission model}} \cdot \left( \sum_{s'} \underbrace{p(z_t = s | z_{t-1}=s')}_{\text{transition model}} \cdot \underbrace{p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s')}_{\text{forward variable for previous timestep}} \right)\\
    &= b_s(x_t) \cdot \left( \sum_{s'} A_{s, s'} \cdot \alpha_{s',t-1} \right)
\end{align}
$

이전 줄에서 이 방정식의 각 줄로 이동하는 방법을 설명하겠습니다.

라인 1은 정방향 변수 $\alpha_{s,t}$의 정의입니다.

2행은 체인 규칙($p(A,B) = p(A|B) \cdot p(B)$, 여기서 $A$는 $x_t$이고 $B$는 다른 모든 변수임)입니다.

3행에서는 가정 2를 적용합니다. 관측 확률 $x_t$는 현재 상태 $z_t$에만 의존합니다.

4행에서는 이전 타임스텝 $t-1$에서 가능한 모든 상태를 주변화합니다.

5행에서 체인 규칙을 다시 적용합니다.

6행에서는 가정 1을 적용합니다. 현재 상태는 이전 상태에만 의존합니다.

7행에서 방출 확률, 전이 확률 및 이전 시간 단계의 순방향 변수를 대체하여 완전한 재귀를 얻습니다.

위 공식은 $t = 2 \rightarrow T$에 사용할 수 있습니다. $t=1$에서는 이전 상태가 없으므로 전이 행렬 $A$ 대신 각 상태에서 시작할 확률을 알려주는 상태 이전 $\pi$를 사용합니다. 따라서 $t=1$의 경우 순방향 변수는 다음과 같이 계산됩니다.

$$\begin{align} 
\alpha_{s,1} &= p(x_1, z_1=s) \\
  &= p(x_1 | z_1 = s) \cdot p(z_1 = s)  \\ 
&= b_s(x_1) \cdot \pi_s
\end{align}$$

마지막으로 $p(\mathbf{x}) = p(x_1, x_2, \dots, x_T)$를 계산하기 위해 마지막 시간 단계에서 계산된 순방향 변수인 $\alpha_{s,T}$를 주변화합니다.

$$\begin{align*} 
p(\mathbf{x}) &= \sum_{s} p(x_1, x_2, \dots, x_T, z_T = s) \\ 
&= \sum_{s} \alpha_{s,T}
\end{align*}$$

정방향 변수의 로그를 취하고 다음 ID를 사용하여 이 공식에서 로그 도메인 공식으로 얻을 수 있습니다.
- $\text{log }(a \cdot b) = \text{log }a + \text{log }b$
- $\text{log }(a + b) = \text{log }(e^{\text{log }a} + e^{\text{log }b}) = \text{logsumexp}(\text{log }a, \text{log }b)$

### 문제 2: $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x})$를 어떻게 계산합니까?

관찰 시퀀스 $\mathbf{x}$가 주어지면 $\mathbf{x}$를 생성할 수 있는 가장 가능성 있는 상태 시퀀스를 찾고자 할 수 있습니다. (셀카 순서가 주어졌을 때 친구가 어떤 도시를 방문했는지 유추하고 싶습니다.) 즉, $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\ mathbf{x})$.

베이즈 규칙을 사용하여 이 식을 다시 작성할 수 있습니다.
$$\begin{align*} 
    \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x}) &= \underset{\mathbf{z}}{\text{argmax }} \frac{p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})}{p(\mathbf{x})} \\ 
    &= \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})
\end{align*}$$

Hmm! 이 마지막 식은은, $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$, forward algorithm을 도입 하기전에 접했던 다루기 힘든 표현과 의심스러울 정도로 유사해 보입니다. $\underset{\mathbf{z}}{\sum} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$.

그리고 실제로 모든 $\mathbf{z}$에 대한 다루기 힘든 *sum*이 순방향 알고리즘을 사용하여 효율적으로 구현될 수 있는 것처럼 이 다루기 힘든 *argmax*도 유사한 분할 정복 알고리즘을 사용하여 효율적으로 구현될 수 있습니다. 전형적인 Viterbi 알고리즘!

________

<u><b>The Viterbi Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,1} := b_s(x_1) \cdot \pi_s$\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,1} := 0$
>
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,t} := b_s(x_t) \cdot \left( \underset{s'}{\text{max }} A_{s, s'} \cdot \delta_{s',t-1} \right)$\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,t} := \underset{s'}{\text{argmax }} A_{s, s'} \cdot \delta_{s',t-1}$
> 
> $z_T^* := \underset{s}{\text{argmax }} \delta_{s,T}$\
> for $t = T-1 \rightarrow 1$:\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_{t}^* := \psi_{z_{t+1}^*,t+1}$
> 
> $\mathbf{z}^* := \{z_{1}^*, \dots, z_{T}^* \}$\
return $\mathbf{z}^*$
________

Viterbi 알고리즘은 순방향 알고리즘보다 다소 형편없어 보이지만 본질적으로 동일한 알고리즘이며 두 가지 조정이 있습니다. 1) 이전 상태에 대한 합계를 취하는 대신 최대값을 취합니다. 2) 이전 상태의 argmax를 테이블에 기록하고 마지막에 이 테이블을 반복하여 가장 가능성이 높은 상태 시퀀스인 $\mathbf{z}^*$를 얻습니다. (그리고 순방향 알고리즘과 마찬가지로 더 나은 수치적 안정성을 위해 로그 도메인에서 Viterbi 알고리즘을 실행해야 합니다.)

Viterbi 알고리즘을 PyTorch 모델에 추가해 보겠습니다.

In [88]:
a = torch.tensor([0.1, 0.9])
out1,out2 = torch.max(a, dim=0)
print(out1,out2)

tensor(0.9000) tensor(1)


In [89]:
a = torch.tensor([[0.1, 0.9],
                  [0.8, 0.2]])
out1,out2 = torch.max(a, dim=1)
print(out1,out2)

tensor([0.9000, 0.8000]) tensor([1, 0])


In [90]:
a = torch.tensor([[0.5, 0.5],
                  [0.8, 0.2]])
out1,out2 = torch.max(a, dim=1)
print(out1,out2)

tensor([0.5000, 0.8000]) tensor([0, 0])


In [96]:
a = [1,2,3]
a.insert(0,4)
print(a)

[4, 1, 2, 3]


In [103]:
def viterbi(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)
  Find argmax_z log p(x|z) for each (x) in the batch.
  """
  if self.is_cuda:
    x = x.cuda()
    T = T.cuda()

  batch_size = x.shape[0]; T_max = x.shape[1]
  log_state_priors = torch.nn.functional.log_softmax(self.unnormalized_state_priors, dim=0)
  log_delta = torch.zeros(batch_size, T_max, self.N).float()
  psi = torch.zeros(batch_size, T_max, self.N).long()
  if self.is_cuda:
    log_delta = log_delta.cuda()
    psi = psi.cuda()

  log_delta[:, 0, :] = self.emission_model(x[:,0]) + log_state_priors
  for t in range(1, T_max):
    max_val, argmax_val = self.transition_model.maxmul(log_delta[:, t-1, :])
    log_delta[:, t, :] = self.emission_model(x[:,t]) + max_val
    psi[:, t, :] = argmax_val

#   print("log_delt", log_delta)
#   print("psi", psi)

  # Get the log probability of the best path
  log_max = log_delta.max(dim=2)[0]
  best_path_scores = torch.gather(log_max, 1, T.view(-1,1) - 1)

  # This next part is a bit tricky to parallelize across the batch,
  # so we will do it separately for each example.
  z_star = []
  for i in range(0, batch_size):
    z_star_i = [ log_delta[i, T[i] - 1, :].max(dim=0)[1].item() ]
    for t in range(T[i] - 1, 0, -1):
      z_t = psi[i, t, z_star_i[0]].item()
      z_star_i.insert(0, z_t)

    z_star.append(z_star_i)

  return z_star, best_path_scores # return both the best path and its log probability

def transition_model_maxmul(self, log_alpha):
  log_transition_matrix = torch.nn.functional.log_softmax(self.unnormalized_transition_matrix, dim=0)

  out1, out2 = maxmul(log_transition_matrix, log_alpha.transpose(0,1))
  return out1.transpose(0,1), out2.transpose(0,1)

def maxmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Similar to the log domain matrix multiplication,
	this computes out_{i,j} = max_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	log_A_expanded = torch.stack([log_A] * p, dim=2)
	log_B_expanded = torch.stack([log_B] * m, dim=0)

	elementwise_sum = log_A_expanded + log_B_expanded
	out1,out2 = torch.max(elementwise_sum, dim=1)

	return out1,out2

TransitionModel.maxmul = transition_model_maxmul
HMM.viterbi = viterbi

모음/자음 HMM이 주어지면 입력 시퀀스에서 Viterbi를 실행해 보십시오.

In [104]:
x = torch.stack( [torch.tensor(encode("cat"))] )
T = torch.tensor([3])
print(model.viterbi(x, T))

([[0, 1, 0]], tensor([[-9.8778]], device='cuda:0'))


In [105]:
x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
T = torch.tensor([3,3])
print(model.viterbi(x, T))

([[1, 0, 1], [1, 0, 0]], tensor([[-8.0370],
        [   -inf]], device='cuda:0'))


$\mathbf{x} = \text{"aba"}$의 경우 Viterbi 알고리즘은 $\mathbf{z}^* = \{1,0,1\}$를 반환합니다. 이것은 위의 상태를 정의한 방식에 따라 "모음, 자음, 모음"에 해당하며, 이 입력 시퀀스에 맞습니다.

$\mathbf{x} = \text{"abb"}$의 경우 Viterbi 알고리즘은 여전히 $\mathbf{z}^*$를 반환하지만 "모음, 자음, 자음"은 이 HMM이고 실제로 이 경로의 로그 확률은 $-\infty$입니다.

"순방향 점수"(순방향 알고리즘에 의해 반환된 모든 가능한 경로의 로그 확률)와 "Viterbi 점수"(Viterbi 알고리즘에 의해 반환된 최대 우도 경로의 로그 확률)를 비교해 보겠습니다.

In [106]:
x = torch.stack( [torch.tensor(encode("cat"))] )
T = torch.tensor([3])
print(model.forward(x, T))
print(model.viterbi(x, T)[1])

tensor([[-9.8778]], device='cuda:0')
tensor([[-9.8778]], device='cuda:0')


두 점수는 동일합니다! 이 경우 HMM을 통과하는 가능한 경로는 하나뿐이므로 가장 가능성이 높은 경로의 확률은 가능한 모든 경로의 확률의 합과 동일하기 때문입니다.

그러나 일반적으로 포워드 스코어와 Viterbi 스코어는 항상 다소 비슷합니다. 이는 $\text{logsumexp}$ 함수의 속성인 $\text{logsumexp}(\mathbf{x}) \approx \max (\mathbf{x})$ 때문입니다. ($\text{logsumexp}$는 "smooth maximum" 함수라고도 합니다.)

In [101]:
x = torch.tensor([1., 2., 3.])
print(x.max(dim=0)[0])
print(x.logsumexp(dim=0))

tensor(3.)
tensor(3.4076)


### 문제 3: 모델을 어떻게 학습시키나요?





이전에는 특정 동작을 갖도록 HMM을 하드 코딩했습니다. 대신 우리가 하고 싶은 것은 HMM이 자체적으로 데이터를 모델링하는 방법을 배우게 하는 것입니다. 상태가 특정 해석을 갖도록 HMM과 함께 지도 학습을 사용할 수 있지만(방출 모델 또는 전환 모델을 하드 코딩하여) HMM의 정말 멋진 점은 자연스럽게 비지도 학습자라는 점입니다. 프로그래머가 각 상태의 의미를 표시할 필요 없이 서로 다른 상태를 사용하여 데이터의 서로 다른 패턴을 나타내는 방법을 배울 수 있습니다.

많은 기계 학습 모델과 마찬가지로 HMM은 다음과 같이 최대 우도 추정을 사용하여 훈련할 수 있습니다.

$$\theta^* = \underset{\theta}{\text{argmin }} -\sum_{\mathbf{x}^i}\text{log }p_{\theta}(\mathbf{x}^i)$$

여기서 $\mathbf{x}^1, \mathbf{x}^2, \dots$는 훈련 예시입니다.

이를 위한 표준 방법은 EM(Expectation-Maximization) 알고리즘이며 HMM의 경우 "Baum-Welch" 알고리즘이라고도 합니다. EM 교육에서는 잠재 변수의 값을 추정하는 "E-단계"와 추정된 잠재 변수에 따라 모델 매개변수가 업데이트되는 "M-단계"를 번갈아 사용합니다. ($k$-각 데이터 포인트가 속한 클러스터를 추측한 다음 클러스터가 있는 위치를 다시 추정하고 반복합니다.) EM 알고리즘에는 몇 가지 좋은 속성이 있습니다. 각 단계에서 손실 함수를 줄이기 위해 보장됩니다. E-step 및 M-step은 정확한 폐쇄형 솔루션을 가질 수 있으며, 이 경우 성가신 학습 속도가 필요하지 않습니다.

그러나 HMM 순방향 알고리즘은 모든 모델 매개변수와 관련하여 미분 가능하기 때문에 PyTorch와 같은 라이브러리에서 자동 미분 방법을 활용하고 $-\text{log }p_{\theta}(\mathbf{ x})$ 직접, 정방향 알고리즘을 통해 역전파하고 확률적 경사 하강법을 실행합니다. 즉, 훈련을 구현하기 위해 추가 HMM 코드를 작성할 필요가 없습니다. `loss.backward()`만 있으면 됩니다.

여기서는 PyTorch에서 HMM에 대한 SGD 교육을 구현합니다. 먼저 일부 도우미 클래스:

In [107]:
import torch.utils.data
from collections import Counter
from sklearn.model_selection import train_test_split

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, lines):
    self.lines = lines # list of strings
    collate = Collate() # function for generating a minibatch from strings
    self.loader = torch.utils.data.DataLoader(self, batch_size=1024, num_workers=1, shuffle=True, collate_fn=collate)

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx):
    line = self.lines[idx].lstrip(" ").rstrip("\n").rstrip(" ").rstrip("\n")
    return line

class Collate:
  def __init__(self):
    pass

  def __call__(self, batch):
    """
    Returns a minibatch of strings, padded to have the same length.
    """
    x = []
    batch_size = len(batch)
    for index in range(batch_size):
      x_ = batch[index]

      # convert letters to integers
      x.append(encode(x_))

    # pad all sequences with 0 to have same length
    x_lengths = [len(x_) for x_ in x]
    T = max(x_lengths)
    for index in range(batch_size):
      x[index] += [0] * (T - len(x[index]))
      x[index] = torch.tensor(x[index])

    # stack into single tensor
    x = torch.stack(x)
    x_lengths = torch.tensor(x_lengths)
    return (x,x_lengths)

학습/테스트 데이터를 로드해 보겠습니다. 기본적으로 이것은 유닉스 "단어" 파일을 사용하지만 자신의 텍스트 파일을 사용할 수도 있습니다.

In [108]:
!wget https://raw.githubusercontent.com/lorenlugosch/pytorch_HMM/master/data/train/training.txt

filename = "training.txt"

with open(filename, "r") as f:
  lines = f.readlines() # each line of lines will have one word

alphabet = list(Counter(("".join(lines))).keys())
train_lines, valid_lines = train_test_split(lines, test_size=0.1, random_state=42)
train_dataset = TextDataset(train_lines)
valid_dataset = TextDataset(valid_lines)

M = len(alphabet)

--2023-01-05 18:10:46--  https://raw.githubusercontent.com/lorenlugosch/pytorch_HMM/master/data/train/training.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2493109 (2.4M) [text/plain]
Saving to: ‘training.txt’

training.txt        100%[===================>]   2.38M  --.-KB/s    in 0.05s   

2023-01-05 18:10:47 (47.6 MB/s) - ‘training.txt’ saved [2493109/2493109]



모델을 훈련하고 테스트하기 위해 Trainer 클래스를 사용합니다.



In [109]:
from tqdm import tqdm # for displaying progress bar

class Trainer:
  def __init__(self, model, lr):
    self.model = model
    self.lr = lr
    self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=0.00001)
  
  def train(self, dataset):
    train_loss = 0
    num_samples = 0
    self.model.train()
    print_interval = 50
    for idx, batch in enumerate(tqdm(dataset.loader)):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      train_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        for _ in range(5):
          sampled_x, sampled_z = self.model.sample()
          print(decode(sampled_x))
          print(sampled_z)
    train_loss /= num_samples
    return train_loss

  def test(self, dataset):
    test_loss = 0
    num_samples = 0
    self.model.eval()
    print_interval = 50
    for idx, batch in enumerate(dataset.loader):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      test_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        sampled_x, sampled_z = self.model.sample()
        print(decode(sampled_x))
        print(sampled_z)
    test_loss /= num_samples
    return test_loss

마지막으로 모델을 초기화하고 기본 교육 루프를 실행합니다. 코드는 배치 50개마다 모델에서 몇 개의 샘플을 생성합니다. 시간이 지남에 따라 이러한 샘플은 점점 더 사실적으로 보일 것입니다.

In [110]:
# Initialize model
model = HMM(N=64, M=M)

# Train the model
num_epochs = 10
trainer = Trainer(model, lr=0.01)

for epoch in range(num_epochs):
        print("========= Epoch %d of %d =========" % (epoch+1, num_epochs))
        train_loss = trainer.train(train_dataset)
        valid_loss = trainer.test(valid_dataset)

        print("========= Results: epoch %d of %d =========" % (epoch+1, num_epochs))
        print("train loss: %.2f| valid loss: %.2f\n" % (train_loss, valid_loss) )

========= Epoch 1 of 10 =========


  0%|          | 1/208 [00:00<00:40,  5.17it/s]

loss: 38.98460388183594
KSHnmdKPMg
[57, 50, 20, 62, 7, 59, 17, 51, 3, 27]
oEbdcE
hFN
[28, 40, 25, 47, 56, 2, 35, 4, 3, 28]
gOTeIVDTyN
[48, 13, 54, 54, 0, 13, 26, 27, 25, 44]
f-OdHrIZ
t
[23, 38, 35, 19, 52, 17, 37, 26, 11, 41]
NThgLCYcwS
[13, 54, 35, 48, 44, 38, 54, 34, 58, 17]


 26%|██▋       | 55/208 [00:01<00:03, 41.08it/s]

loss: 33.52580261230469
tTcaQRJIOV
[6, 63, 19, 3, 10, 0, 56, 43, 35, 62]
vatstCARah
[41, 24, 29, 39, 62, 17, 32, 45, 13, 0]
SDWynShPBi
[60, 63, 42, 46, 33, 24, 48, 38, 21, 42]
kfYDTr-MHm
[27, 53, 20, 24, 55, 20, 38, 60, 33, 8]
kRoSsUnul-
[29, 49, 16, 24, 11, 10, 0, 56, 24, 27]


...

loss: 23.28838539123535
setewaneju
[25, 62, 18, 62, 12, 8, 48, 62, 0, 56]
tymnarmoco
[18, 11, 20, 48, 8, 23, 55, 46, 51, 46]
apanglylea
[6, 57, 8, 63, 53, 42, 11, 42, 62, 8]
Binnolenca
[30, 29, 63, 43, 46, 48, 62, 63, 18, 6]
nochundest
[30, 46, 18, 32, 29, 63, 43, 62, 24, 19]


 99%|█████████▊| 205/208 [00:04<00:00, 38.64it/s]

loss: 22.647130966186523
fusulmicth
[40, 56, 25, 56, 22, 55, 33, 24, 18, 32]
scelliwerc
[25, 18, 37, 22, 42, 62, 12, 37, 23, 18]
bronickult
[34, 10, 46, 48, 33, 51, 58, 37, 23, 18]
mabilmazon
[44, 8, 21, 61, 22, 55, 8, 59, 37, 63]
sadichiccc
[25, 6, 36, 33, 51, 32, 61, 51, 51, 19]


100%|██████████| 208/208 [00:05<00:00, 41.20it/s]


loss: 23.05544662475586
dianglenno
[13, 33, 8, 63, 53, 42, 62, 63, 48, 46]
========= Results: epoch 10 of 10 =========
train loss: 23.25| valid loss: 23.27



$N$의 다른 값을 시도하고 샘플 품질에 미치는 영향을 확인할 수 있습니다.

In [112]:
x = torch.tensor(encode("quack")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T))

x = torch.tensor(encode("quick")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T))

x = torch.tensor(encode("qurck")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T)) # should have lower probability---in English only vowels follow "qu"

x = torch.tensor(encode("qiick")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T)) # should have lower probability---in English only "u" follows "q"


([[0, 56, 8, 51, 58]], tensor([[-13.8923]], device='cuda:0', grad_fn=<GatherBackward0>))
([[0, 56, 61, 51, 58]], tensor([[-14.0026]], device='cuda:0', grad_fn=<GatherBackward0>))
([[0, 56, 23, 51, 58]], tensor([[-15.7310]], device='cuda:0', grad_fn=<GatherBackward0>))
([[0, 56, 61, 51, 58]], tensor([[-20.0309]], device='cuda:0', grad_fn=<GatherBackward0>))


## 결론

HMM은 자연어 처리에서 매우 인기가 있는 RNN 및 트랜스포머와 신경망 모델에 의해 크게 가려졌습니다. HMM을 공부하는 것은 애매하고 긴장합니다. [Connectionist Temporal Classification](https://www.cs.toronto.edu/~graves/icml_2006.pdf)과 일반적으로 사용되는 일부 기계 학습 기술은 HMM 방법에서 영감을 받았습니다. HMM은 [여전히 인식에서 신경망과 함께 사용](https://arxiv.org/abs/1811.07453)합니다. 여기서 원-핫상태의 가정은 한 번에 하나씩 주장하는 음소에 만족합니다.